In [ ]:
import os
import sys
import json

import warnings

import numpy as np
from scipy import signal
import librosa

sys.path.insert(0, '/mnt/d/projects/bassline_extraction') 

import utilities as utils
from plotting import *
from transcription import *
from signal_processing import *

warnings.filterwarnings('ignore')
%matplotlib inline
np.set_printoptions(suppress=True)

# Initialization

## Directories

**TO DO: aligned_beat_positions => progression_beat_positions**

In [ ]:
with open('../data/ouz_tracks.txt', 'r') as infile:
    track_titles = infile.read().split('\n')
       
_,_, bad_examples = next(os.walk('../data/bassline_extraction/beat_grid/bad_examples'))
bad_examples = [title.split('.txt')[0] for title in bad_examples]
       
with open('../data/metadata/scales_frequencies.json','r') as infile:
    scales = json.load(infile)

with open('../data/metadata/TechHouse_track_dicts.json','r') as infile:
    track_dicts = json.load(infile) 

def pYIN_F0(audio, fs, frame_length):

    hop_length = int(frame_length/4) # 4 by default

    F0, _, confidence = pyin(audio,
                            sr=fs,
                            frame_length=frame_length,
                            hop_length=hop_length,
                            fmin=31.0,
                            fmax=130.0,
                            fill_na=0.0)
    
    plot_confidence(title, confidence, save=True, plot_title='Confidence_pYIN')

    time_axis = np.arange(len(F0)) * (hop_length/fs)

    return (time_axis, F0)#, (time_axis, F0_filtered)

In [ ]:
def pYIN_F0(audio, fs, frame_length, threshold='none'): #, win_length, hop_length, threshold='none'):
    
    hop_length = int(frame_length/4) # 4 by default
    
    F0, _, confidence = pyin(audio,
                            sr=fs,
                            frame_length=frame_length,
                            win_length=win_length,
                            #hop_length=hop_length,
                            #resolution=0.5,
                            #boltzmann_parameter = 100,
                            fmin=31.0,
                            fmax=130.0,
                            fill_na=0.0)

    mean, std = np.mean(confidence), np.std(confidence)
    
    plot_confidence(title, confidence, save=True, plot_title='Confidence_pYIN')

    if threshold == 'none':
        threshold = 0
    elif threshold == 'mean':
        threshold = mean
    elif threshold == 'mean_reduced':
        threshold = mean - (std/2)
    else:
        assert threshold <= 1.0 and threshold > 0, 'Threshold must be inside (0, 1)'

    F0_filtered = np.array(confidence_filter(F0, confidence, threshold))

    time_axis = np.arange(len(F0)) * (hop_length/fs)

    return (time_axis, F0), (time_axis, F0_filtered)

In [ ]:
for title in track_titles:

    try:

        print('{}\n'.format(title))
        BPM = float(track_dicts[title]['BPM'])
        beat_length = 60/BPM

        if title in bad_examples:
            with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
                dpqb = float(infile.read())

            #if dpqb > 
                #continue
            print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))

        notes, scale_frequencies = utils.get_track_scale(title, track_dicts, scales)

        fs = 44100
        chorus, bassline, _ = utils.load_audio(title) # read the chorus and the bassline

        # spectrogram 
        beat_factor = 8
        win_length = int((beat_length/beat_factor)*fs) 
        n_fft = 4096*8
        hop_length = int(win_length/2) 
        assert win_length < n_fft, 'Window length must be greater than N_fft'

        bassline_spectrogram = extract_dB_spectrogram(bassline, n_fft, win_length, hop_length)

        # F0 estimation  
        beat_factor = 4 # quarter 4, 1/8th 8...
        frame_length = int((beat_length/beat_factor)*fs)
        #win_length = int(frame_length/2)
        #hop_length = int(frame_length/8)

        F0_estimate, pitch_track = pYIN_F0(bassline, fs, frame_length, threshold=0.05)

        plot_confidence_filtering_effect(title, bassline_spectrogram, fs, hop_length,
                                         F0_estimate, pitch_track, save=True, plot_title='')
            
    except KeyboardInterrupt:
        import sys
        sys.exit()
        pass 
    except:
        print('error on {}'.format(title))

## Sİngle track

In [ ]:
title = track_titles[6]

print('{}\n'.format(title))
BPM = float(track_dicts[title]['BPM'])
beat_length = 60/BPM

if title in bad_examples:
    with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
        dpqb = float(infile.read())

    #if dpqb > 
        #continue
    print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))

notes, scale_frequencies = utils.get_track_scale(title, track_dicts, scales)

fs = 44100
chorus, bassline, _ = utils.load_audio(title) # read the chorus and the bassline

# spectrogram 
beat_factor = 8
win_length = int((beat_length/beat_factor)*fs) 
n_fft = 4096*8
hop_length = int(win_length/2) 
assert win_length < n_fft, 'Window length must be greater than N_fft'

bassline_spectrogram = extract_dB_spectrogram(bassline, n_fft, win_length, hop_length)

# F0 estimation  
beat_factor = 4 # quarter 4, 1/8th 8...
frame_length = int((beat_length/beat_factor)*fs)
#win_length = int(frame_length/2)
#hop_length = int(frame_length/8)

F0_estimate, pitch_track = pYIN_F0(bassline, fs, frame_length, threshold=0.05)

plot_confidence_filtering_effect(title, bassline_spectrogram, fs, hop_length,
                                 F0_estimate, pitch_track, save=True, plot_title='')

## Crepe vs pYIN

In [ ]:
for i in np.random.randint(0, 200, 50):
    
    try:
        
        title = track_titles[i]
        print('{}\n'.format(title))
        BPM = float(track_dicts[title]['BPM'])
        beat_length = 60/BPM

        if title in bad_examples:
            with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
                dpqb = float(infile.read())
            print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))

        notes, scale_frequencies = utils.get_track_scale(title, track_dicts, scales)

        fs = 44100
        chorus, bassline, unprocessed_bassline = utils.load_audio(title) # read the chorus and the bassline

        beat_factor = 4
        win_length = int((beat_length/beat_factor)*fs) 

        n_fft = 4096*8
        #win_length = 4096*4

        hop_length = int(win_length/2) 
        assert win_length < n_fft, 'Window length must be greater than N_fft'

        #chorus_spectrogram = extract_dB_spectrogram(chorus, n_fft, win_length, hop_length)
        bassline_spectrogram = extract_dB_spectrogram(bassline, n_fft, win_length, hop_length)


        beat_factor = 8 
        F0_estimates  = yin_crepe(title, bassline, fs, int((beat_length/beat_factor)*fs), save=True) 

        plot_algorithm_comparison_raw(title, bassline_spectrogram, fs, hop_length, F0_estimates, ['CREPE', 'pYIN'], save=True)
        
    except:
        print(title)

In [ ]:
def yin_crepe(title, bassline, fs, frame_length, save=False):
        
    time_axis, F0, confidence_crepe, _ = crepe_predict(bassline, fs, viterbi=True)
           
    #F0_filtered = np.array(confidence_filter(F0, confidence_crepe, threshold))
    
    F0_estimate_crepe = (time_axis, F0)
    #pitch_track_crepe = (time_axis, F0_filtered)
    
    
    hop_length = int(frame_length/4) # 4 by default   
    F0, _, confidence_pyin = pyin(bassline, sr=fs, frame_length=frame_length, fmin=31.0, 
                            fmax=130.0, fill_na=0.0)

    #F0_filtered = np.array(confidence_filter(F0, confidence_pyin, threshold))
    
    time_axis = np.arange(len(F0)) * (hop_length/fs) 
    
    F0_estimate_pyin = (time_axis, F0)
    #pitch_track_pyin = (time_axis, F0_filtered) 
    
      
    plot_compared_confidences(title, confidence_crepe, confidence_pyin, save=save)
    
    return [F0_estimate_crepe, F0_estimate_pyin] #, [pitch_track_crepe, pitch_track_pyin]

In [ ]:
beat_factor = 8 
F0_estimates  = yin_crepe(title, bassline, fs, int((beat_length/beat_factor)*fs), save=True)

In [ ]:
plot_algorithm_comparison_raw(title, bassline_spectrogram, fs, hop_length, F0_estimates, ['CREPE', 'pYIN'], save=True)